# 04 - Amazon SageMaker jobs and pipeline for documents processing

Steps of the pipeline:

If you want to run the pipeline on new documents, run the the steps 1 to 4 below, then run this notebook.
Alternatively, if you simply want to demo or explore this solution run this notebook which will upload pre-created data automatically
if no data already exists.

1. Download documents: use `experiments/02-download-raw-pdf-documents.ipynb`
2. Parse out the text: use `experiments/03-pdf-document-processing.ipynb`
3. Create embedding and semantic search index: use `experiments/04-question-answering-using-rag.ipynb`
4. Extract entities: use `experiments/05-entities-extraction-to-structured-metadata.ipynb`
5. Load embeddings into PostgreSQL: first step of the pipeline below.
6. Load entities into SQL table: second stpe of the pipeline below.


Run this notebook after running notebooks 01 to 05 under `experiments`. This will execute a SageMaker pipeline that prepares the embeddings and SQL tables and loads them into the PostgreSQL database.

In [2]:
import sys

import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
pipeline_session = PipelineSession()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
import json

ssm = boto3.client("ssm")
secretsmanager = boto3.client("secretsmanager")
region = boto3.session.Session().region_name

In [4]:
security_group_parameter = "/AgenticLLMAssistant/SMProcessingJobSecurityGroupId"
dbsecret_arn_parameter = "/AgenticLLMAssistant/DBSecretARN"
subnet_ids_parameter = "/AgenticLLMAssistant/SubnetIds"
s3_bucket_name_parameter = "/AgenticLLMAssistant/AgentDataBucketParameter"

security_group = ssm.get_parameter(Name=security_group_parameter)
security_group = security_group["Parameter"]["Value"]

db_secret_arn = ssm.get_parameter(Name=dbsecret_arn_parameter)
db_secret_arn = db_secret_arn["Parameter"]["Value"]

subnet_ids = ssm.get_parameter(Name=subnet_ids_parameter)
private_subnets_with_egress_ids = json.loads(subnet_ids["Parameter"]["Value"])

s3_bucket_name = ssm.get_parameter(Name=s3_bucket_name_parameter)
s3_bucket_name = s3_bucket_name["Parameter"]["Value"]


In [5]:
processed_documents_s3_key = "documents_processed.json"
sql_tables_s3_key = "structured_metadata"

In [6]:
!aws s3 ls {s3_bucket_name}/{processed_documents_s3_key}

2024-03-15 13:56:13      67716 documents_processed.json


In [8]:
!aws s3 ls {s3_bucket_name}/{sql_tables_s3_key}/

2024-03-15 14:56:02       4722 extracted_entities.csv


## Upload pre-created data if no data exists

In [18]:
import boto3
import botocore

# Create an S3 client
s3 = boto3.client('s3')


def upload_file_to_s3_if_doesnt_exist(file_path, bucket_name, s3_file_key):
    try:
        # Try to retrieve the file metadata
        s3.head_object(Bucket=bucket_name, Key=s3_file_key)
        print(f"File '{s3_file_key}' already exists in bucket '{bucket_name}'")
    except botocore.exceptions.ClientError as e:
        # If the error code is 404 (Not Found), the file doesn't exist
        if e.response['Error']['Code'] == '404':
            try:
                # Upload the file to S3
                s3.upload_file(file_path, bucket_name, s3_file_key)
                print(f"File '{s3_file_key}' uploaded to bucket '{bucket_name}'")
            except botocore.exceptions.ClientError as e:
                print(f"Error uploading file: {e}")
        else:
            # Handle other errors, e.g., access denied, bucket doesn't exist, etc.
            print(f"Error occurred: {e}")

Upload Amazon Financial reports pre-extracted with Amazon Textract using the notebooks 1 to 5 referred at the beginning of this notebook.

In [23]:
file_path = "data/documents_processed.json"
s3_file_key = processed_documents_s3_key
upload_file_to_s3_if_doesnt_exist(file_path, s3_bucket_name, s3_file_key)

File 'documents_processed.json' already exists in bucket 'assistantbackendstack-agentdatabucket67afdfb9-pqa1faq3bool'


Upload entities of interest extracted from Amazon Financial reports using notebooks 1 to 5 referred at the beginning of this notebook.

In [25]:
file_path = "data/extracted_entities.csv"
s3_file_key = f"{sql_tables_s3_key}/extracted_entities.csv"
upload_file_to_s3_if_doesnt_exist(file_path, s3_bucket_name, s3_file_key)

File 'structured_metadata/extracted_entities.csv' already exists in bucket 'assistantbackendstack-agentdatabucket67afdfb9-pqa1faq3bool'


## Add pipeline step to prepare and load embeddings

In [27]:
from sagemaker.network import NetworkConfig

# Note if you enable network isolation, with enable_network_isolation=True
# the pip installation of the dependencies
# under scripts/requirements.txt won't work.
current_network_config = NetworkConfig(
    subnets=private_subnets_with_egress_ids, security_group_ids=[security_group]
)

In [28]:
%%time
from sagemaker.xgboost import XGBoostProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role
from sagemaker.workflow.steps import ProcessingStep

# Initialize the XGBoostProcessor
embeddings_and_index_job = XGBoostProcessor(
    framework_version="1.7-1",
    role=get_execution_role(),
    instance_type="ml.m5.large",
    instance_count=1,
    base_job_name="PrepareAndLoadEmbeddingsJob",
    env={
        "SQL_DB_SECRET_ID": db_secret_arn,
        # region used by botocore.
        "AWS_DEFAULT_REGION": region,
    },
    network_config=current_network_config,
    sagemaker_session=pipeline_session
)

# Run the processing job
embeddings_creation_and_indexing_args = embeddings_and_index_job.run(
    code="prepare_and_load_embeddings.py",
    source_dir="scripts",
    inputs=[
        ProcessingInput(
            input_name="processed_documents",
            source=f"s3://{s3_bucket_name}/{processed_documents_s3_key}",
            destination="/opt/ml/processing/input/processed_documents",
        )
    ],
)

embeddings_creation_and_indexing_step = ProcessingStep(
    name="EmbeddingCreationAndIndexing",
    step_args=embeddings_creation_and_indexing_args
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
CPU times: user 105 ms, sys: 0 ns, total: 105 ms
Wall time: 130 ms


/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:297: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


## Add pipeline step to load extract entities into a SQL table

In [29]:
%%time
from sagemaker.xgboost import XGBoostProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

# Initialize the XGBoostProcessor
entities_to_sql = XGBoostProcessor(
    framework_version="1.7-1",
    role=get_execution_role(),
    instance_type="ml.m5.large",
    instance_count=1,
    base_job_name="LoadEntitiesToSQLTableJob",
    env={"SQL_DB_SECRET_ID": db_secret_arn, "AWS_DEFAULT_REGION": region},
    network_config=current_network_config,
    sagemaker_session=pipeline_session,
)

# Run the processing job
load_entities_to_sql_table_args = entities_to_sql.run(
    code="load_sql_tables.py",
    source_dir="scripts",
    inputs=[
        ProcessingInput(
            input_name="sqltables",
            source=f"s3://{s3_bucket_name}/{sql_tables_s3_key}",
            destination="/opt/ml/processing/input/sqltables",
        )
    ]
)

load_entities_to_sql_table_step = ProcessingStep(
    name="LoadEntitiesToSQLTable",
    step_args=load_entities_to_sql_table_args
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
CPU times: user 59.1 ms, sys: 0 ns, total: 59.1 ms
Wall time: 69.6 ms


## Define the SageMaker Pipeline

In [30]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = f"agentic-assistant-data-processing"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[],
    steps=[
        embeddings_creation_and_indexing_step,
        load_entities_to_sql_table_step
    ]
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


print the pipeline definition in json

In [31]:
import json


definition = json.loads(pipeline.definition())
definition

INFO:sagemaker.processing:Uploaded scripts to s3://sagemaker-eu-west-1-AWS_ACCOUNT_ID/agentic-assistant-data-processing/code/9467369aa148a44f332dd981afda32de/sourcedir.tar.gz


Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-west-1-AWS_ACCOUNT_ID/agentic-assistant-data-processing/code/24c014389a99c012dd3121b9fc882ee9/runproc.sh


{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'EmbeddingCreationAndIndexing',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.m5.large',
      'InstanceCount': 1,
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '141502667606.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-xgboost:1.7-1',
     'ContainerEntrypoint': ['/bin/bash',
      '/opt/ml/processing/input/entrypoint/runproc.sh']},
    'RoleArn': 'arn:aws:iam::AWS_ACCOUNT_ID:role/service-role/AmazonSageMaker-ExecutionRole-20240131T120658',
    'ProcessingInputs': [{'InputName': 'processed_documents',
      'AppManaged': False,
      'S3Input': {'S3Uri': 's3://assistantbackendstack-agentdatabucket67afdfb9-pqa1faq3bool/documents_processed.json',
       'LocalPath': '/opt/ml/processing/input/p

Update or insert the pipeline

In [32]:
pipeline.upsert(role_arn=role)

INFO:sagemaker.processing:Uploaded scripts to s3://sagemaker-eu-west-1-AWS_ACCOUNT_ID/agentic-assistant-data-processing/code/94648ce6e344f7078372ffdc67507a25/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-west-1-AWS_ACCOUNT_ID/agentic-assistant-data-processing/code/ef2f8dbf6c1c58080b9ce991f1318fd8/runproc.sh
INFO:sagemaker.processing:Uploaded scripts to s3://sagemaker-eu-west-1-AWS_ACCOUNT_ID/agentic-assistant-data-processing/code/9467369aa148a44f332dd981afda32de/sourcedir.tar.gz


Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-west-1-AWS_ACCOUNT_ID/agentic-assistant-data-processing/code/24c014389a99c012dd3121b9fc882ee9/runproc.sh
INFO:sagemaker.processing:Uploaded scripts to s3://sagemaker-eu-west-1-AWS_ACCOUNT_ID/agentic-assistant-data-processing/code/94648ce6e344f7078372ffdc67507a25/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-west-1-AWS_ACCOUNT_ID/agentic-assistant-data-processing/code/ef2f8dbf6c1c58080b9ce991f1318fd8/runproc.sh
INFO:sagemaker.processing:Uploaded scripts to s3://sagemaker-eu-west-1-AWS_ACCOUNT_ID/agentic-assistant-data-processing/code/9467369aa148a44f332dd981afda32de/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-west-1-AWS_ACCOUNT_ID/agentic-assistant-data-processing/code/24c014389a99c012dd3121b9fc882ee9/runproc.sh


Using provided s3_resource
Using provided s3_resource


{'PipelineArn': 'arn:aws:sagemaker:eu-west-1:AWS_ACCOUNT_ID:pipeline/agentic-assistant-data-processing',
 'ResponseMetadata': {'RequestId': '6f71177b-2896-402d-af54-d1cc8ac16564',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6f71177b-2896-402d-af54-d1cc8ac16564',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '101',
   'date': 'Sat, 16 Mar 2024 12:46:19 GMT'},
  'RetryAttempts': 0}}

Start the pipeline execution

In [33]:
execution = pipeline.start()

Monitor the pipeline execution.

In [34]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:eu-west-1:AWS_ACCOUNT_ID:pipeline/agentic-assistant-data-processing',
 'PipelineExecutionArn': 'arn:aws:sagemaker:eu-west-1:AWS_ACCOUNT_ID:pipeline/agentic-assistant-data-processing/execution/p4ebbwo6w1qa',
 'PipelineExecutionDisplayName': 'execution-1710593187781',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'agentic-assistant-data-processing',
  'TrialName': 'p4ebbwo6w1qa'},
 'CreationTime': datetime.datetime(2024, 3, 16, 12, 46, 27, 722000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 3, 16, 12, 46, 27, 722000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:eu-west-1:AWS_ACCOUNT_ID:user-profile/d-qv9xqffdqndh/default-20240131t120657',
  'UserProfileName': 'default-20240131t120657',
  'DomainId': 'd-qv9xqffdqndh',
  'IamIdentity': {'Arn': 'arn:aws:sts::AWS_ACCOUNT_ID:assumed-role/AmazonSageMaker-ExecutionRole-20240131T120658/SageMaker',
   'PrincipalId': 'AROAZBD

In [35]:
execution.list_steps()

[{'StepName': 'LoadEntitiesToSQLTable',
  'StartTime': datetime.datetime(2024, 3, 16, 12, 46, 29, 190000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'AttemptCount': 1,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:eu-west-1:AWS_ACCOUNT_ID:processing-job/pipelines-p4ebbwo6w1qa-LoadEntitiesToSQLTab-RSxpIMTKPw'}}},
 {'StepName': 'EmbeddingCreationAndIndexing',
  'StartTime': datetime.datetime(2024, 3, 16, 12, 46, 29, 190000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'AttemptCount': 1,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:eu-west-1:AWS_ACCOUNT_ID:processing-job/pipelines-p4ebbwo6w1qa-EmbeddingCreationAnd-Be3C8o4fyD'}}}]